In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload

In [ ]:
import pandas as pd
from pandas import DataFrame
from data_processing import HeartRateDataProcessor, StepProcessor
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pathlib
from os.path import join
# print(pathlib.Path().resolve())

METIME_PATH = join(pathlib.Path().resolve(), "data\\ME-TIME")
METIME_PATH

In [ ]:
from data_processing.data_parsers import me_time_parser

data: dict[int, DataFrame] = me_time_parser(METIME_PATH)

data

In [ ]:
from datetime import datetime
for k in data.keys():    
    data[k].fillna(0, inplace = True)
    data[k] = data[k].astype({"hr": "float", "steps": "float"})
    data[k]["hr"] = (data[k]["hr"] - data[k]["hr"].mean()) / data[k]["hr"].std()
    data[k]["steps"] = (data[k]["steps"] - data[k]["steps"].mean()) / data[k]["steps"].std()


In [ ]:


train = DataFrame()
eval = DataFrame()
test = DataFrame()
for k, d in data.items():
    heartRateProcessor: HeartRateDataProcessor = HeartRateDataProcessor(d)
    heartRateProcessor.process().add_resting_hr()

    stepProcessor: StepProcessor = StepProcessor(d)
    stepProcessor.process()
    
    features = heartRateProcessor.processed.join(stepProcessor.processed)
    features.reset_index(inplace = True)

    features.to_csv(METIME_PATH + "\\features\\" + str(k) + ".csv", index = False)
    print("done " + str(k))
    

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN

In [ ]:
aux = pd.read_csv(METIME_PATH + "\\features\\" + str(1) + ".csv")
aux.fillna(0, inplace = True)
dbscan = DBSCAN(eps=6, min_samples=10).fit(aux.loc[:, aux.columns != 'window'])

labels = dbscan.labels_

print(labels)

# data[99]["hour"] = data[99]["time"].dt.hour
# data[99]["active"] = (data[99]["steps"] != 0)

# a = data[99].groupby("active")["hr"].mean()
# print(a)
# a.plot()
# plt.plot(a["hour"], a["hr"])
# plt.xlabel("time")
# plt.ylabel("HR")

In [ ]:
for k in data.keys():
    # a = data[k]["time"].astype(str)
    df = data[k][(data[k]['time'].dt.month == 6) & 
        (data[k]['time'].dt.year == 2021) & (data[k]['time'].dt.day == 1)]
    l = df['time'].values
    print(l)
    # print(l.astype("datetime64[Y]").astype(int) + 1970)
    a = (np.diff(l) / 1000000000).astype(int)
    # a = a[a != 5]
    print(a[a > 5].size)



    # plt.plot(df["time"].values, df["hr"].values)
    # plt.xlabel("time")
    # plt.ylabel("HR")
    # plt.show()
    # # print(df[df["steps"] == df["steps"].max()])
    # # print(df[df['time'].dt.second == 0][df['steps'] != 0])
    # print(df)
    # print(df)
    
    break

    ts = pd.to_datetime(str(data[k]["time"][0]))
    ts = ts.to_pydatetime()
    print(ts)
    print(type(ts))
    break

In [ ]:
ts = pd.to_datetime(str(data[1]["time"][0]))
print(ts)
ts = ts.to_pydatetime()
print(ts)
print(type(ts))